# Import Libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Parsing 

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,"lxml")


# Table on wikipedia

In [3]:
table=soup.find('table')
col_value=table.find_all('td')
#print(col_value)
count_ele=len(col_value)


In [4]:
postcode=[]
borough=[]
neighbourhood=[]


In [6]:
for i in range(0,count_ele,3):
    postcode.append(col_value[i].text.strip())
    borough.append(col_value[i+1].text.strip())
    neighbourhood.append(col_value[i+2].text.strip())

In [15]:
df_post = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_post.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [16]:
df_post.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Parkwoods
3,M3A,North York,Victoria Village
4,M4A,North York,Harbourfront


In [17]:
df_post.drop(df_post[df_post['Borough'] == 'Not assigned'].index, inplace=True)
df_post.loc[df_post.Neighbourhood == 'Not assigned', "Neighbourhood"] = df_post.Borough

In [18]:
df_post.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Victoria Village
4,M4A,North York,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M5A,Downtown Toronto,Lawrence Heights
7,M6A,North York,Lawrence Manor


# Grouping the data

In [20]:
df_grp = df_post.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_grp.columns = ['Postcode', 'Borough', 'Neighbourhood']

# The First Frame


In [22]:
df_grp.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Scarborough"
1,M1C,Scarborough,"Rouge Hill, Port Union, Scarborough"
2,M1E,Scarborough,"Morningside, West Hill, Scarborough"
3,M1G,Scarborough,Scarborough
4,M1H,Scarborough,Hillcrest Village


# Now dealing with latitude and longitude

In [25]:
df_latlng=pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df_latlng.columns=['Postcode','Latitude','Longitude']

In [27]:
df_join=pd.merge(df_grp,df_latlng,on=['Postcode'],how='inner')
df_join

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Scarborough",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Scarborough",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill, Scarborough",43.763573,-79.188711
3,M1G,Scarborough,Scarborough,43.770992,-79.216917
4,M1H,Scarborough,Hillcrest Village,43.773136,-79.239476
5,M1J,Scarborough,Fairview,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, Bayview Village",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Silver Hills",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Newtonbrook",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Willowdale South",43.692657,-79.264848
